In [1]:
# %load basic.py
from IPython.display import set_matplotlib_formats, display
import numpy as np
import pandas as pd
import graphviz
import os
import mglearn
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc, font_manager
from cycler import cycler

font_name = font_manager.FontProperties(fname = 'C:/Windows/Fonts/H2GTRM.TTF').get_name()
rc('font', family = font_name)
matplotlib.rcParams['axes.unicode_minus'] = False

set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300
plt.rcParams['image.cmap'] = "viridis"
plt.rcParams['image.interpolation'] = "none"
plt.rcParams['savefig.bbox'] = "tight"
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['legend.numpoints'] = 1
plt.rc('axes', prop_cycle=(
    cycler('color', mglearn.plot_helpers.cm_cycle.colors) +
    cycler('linestyle', ['-', '-', "--", (0, (3, 3)), (0, (1.5, 1.5))])))

np.set_printoptions(precision=3, suppress=True)

pd.set_option('display.max_columns', 8)
pd.set_option('precision', 2)

__all__ = ['np', 'mglearn', 'display', 'plt', 'pd']

C:\Users\hyooo\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
C:\Users\hyooo\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# 7.1 문자열 데이터 타입

문자열 데이터 종류  
- 범주형 데이터
- 범주에 의미를 연결시킬 수 있는 임의의 문자열
- 구조화된 문자열 데이터
- 텍스트 데이터

- **범주형 데이터**는 고정된 목록으로 구성됩니다.  
(ex 1) 사람들에게 가장 좋아하는 색을 묻는 설문으로 데이터 수집(드롭다운 메뉴)  
    데이터셋에 '빨강', '녹생', '파랑' 등 8개의 값 중 하나가 들어가며 당연히 범주형 변수로 인코딩됨  
    -> 고유한 값을 찾아 얼마나 자주 나타나는지 히스토그램을 그려볼 수 있음  
    -> 각 값이 어플리케이션에서 생각하는 범주에 적절한지 확인  
    -> 예를 들어 '검정'을 '검점'으로 쓴 오타가 있다면 같은 의미이므로 두 값을 하나로 합쳐야 함  
    

- 텍스트 필드로 받는 응답은 **범주에 의미를 연결시킬 수 있는 임의의 문자열**에 해당합니다.  
(ex 2) 사람들에게 가장 좋아하는 색을 묻는 설문으로 데이터 수집(텍스트 필드)  
    -> 철자가 틀리거나 '회색', '쥐색'처럼 다르게 쓸 수 있음  
    -> 매우 이상한 이름을 입력받을 수도 있음   
    
    
- 미리 정의된 범주에 속하지 않지만 직접 입력한 값들이 주소나 장소, 사람 이름, 날짜, 전화번호, 식별번호처럼 일정한 **구조**를 가지기도 함  


- 자유로운 형태의 절과 문장으로 구성된 **텍스트 데이터**는 트위터, 채팅, 호텔 리뷰, 셰익스피어 작품, 위키백과 문서 등이 있습니다.  
    -> 대부분 단어로 구성된 문장에 정보를 담고 있음  
        (데이터셋 : 말뭉치, 하나의 텍스트를 의미하는 각 데이터 포인트 : 문서)

# 7.2 예제 어플리케이션 : 영화 리뷰 감성 분석